In [ ]:
#You need to run the photometry analysis before you are ready for this step

#In this script we will use the deeplabcut data to determine nose-to-nose sniffing, and the angle between
#the test mouse and the target mouse

In [ ]:
import pandas as pd
import numpy as np

# Import DLC data
data = pd.read_csv('Your File Path/deeplabcut_file.csv', skiprows=[0,1,2], header=None)

# Rename columns
data.drop(labels=[0,1,2], axis=0, inplace=True)
data.columns = ['frames','nose1_X','nose1_Y','likelihood_n1','right_ear1_X','right_ear1_Y','likelihood_re1',
                'left_ear1_X','left_ear1_Y','likelihood_le1','midpoint1_X','midpoint1_Y','likelihood_mp1',
                'right_leg1_X','right_leg1_Y','likelihood_rl1','left_leg1_X','leg_leg1_Y','likelihood_ll1',
               'tailbase1_X','tailbase1_Y','likelihood_tb1','tailtip1_X','tailtip1_Y','likelihood_tt1',
               'nose2_X', 'nose2_Y','likelihood_n2','right_ear2_X','right_ear2_Y','likelihood_re2',
                'left_ear2_X','left_ear2_Y','likelihood_le2','midpoint2_X','midpoint2_Y','likelihood_mp2',
                'right_leg2_X','right_leg2_Y','likelihood_rl2','left_leg2_X','leg_leg2_Y','likelihood_ll2',
               'tailbase2_X','tailbase2_Y','likelihood_tb2','tailtip2_X','tailtip2_Y','likelihood_tt2']
data = data.astype('float')

# Convert frames to seconds
time = data['frames'] / 30

#Note: 23 pixels = 1 cm in the uncaged SI arena

# Nose-to-nose Sniff
NN_sniff = np.sqrt(((data['nose1_X'] - data['nose2_X']) ** 2) + ((data['nose1_Y'] - data['nose2_Y']) ** 2)) #find the distance between the noses of the 2 mice
NN_sniff = NN_sniff <=46 #46 pixels = 2cm
NN_sniff = NN_sniff.replace({True: 1, False: 0}) #column = 1 when NN sniff, column = 0 when no NN sniiff
NN_sniff = NN_sniff.reset_index(drop=True)
NN_sniff[0:900] = 0 #target mouse enters 30 sec into test so set all values = 0 for the 1st 30 sec

# Nose-to-nose Sniff Interaction Starts Only
bool_array = NN_sniff
NN_new = np.zeros((len(bool_array),)) #create an array of zeros the length of the data

for i in range(900,len(bool_array)): #we are going to change the value of the row from 0 to 1 to denote the start of a NN sniff bout
    if bool_array[i] == 1:
        if np.sum(bool_array[i-60:i]) == 0: #if there has been a 1 in the last 60 frames (2 sec) do not score as 1
            if np.sum(bool_array[i:i+6]) >= 3: #score as 1 if there are at least 3 instances of NN Sniff within the next 6 frames
                NN_new [i] = 1
                
NN_new = pd.Series(NN_new)

# calculate proximity between mice
prox = np.sqrt(((data['midpoint1_X'] - data['midpoint2_X']) ** 2) + ((data['midpoint1_Y'] - data['midpoint2_Y']) ** 2))
prox = prox <=184 #184 pixels = 8cm
prox = prox.replace({True: 1, False: 0})
prox = prox.reset_index(drop=True)
prox[0:900] = 0

# calculate angle between mice
x2, y2 = np.array([data['nose1_X'],data['nose1_Y']])
x1, y1 = np.array([data['midpoint1_X'],data['midpoint1_Y']])
x3, y3 = np.array([data['nose2_X'],data['nose2_Y']])
radian = np.arctan2(y2-y1, x2-x1) - np.arctan2(y3-y1, x3-x1)

radian = np.arctan2(y2-y1, x2-x1) - np.arctan2(y3-y1, x3-x1)
angle = np.abs(radian*180/np.pi)
angle = [i - 360 if i > 180 else i for i in angle]
angle = pd.Series(angle)
angle = abs(angle)
angle[0:900] = np.nan

# angle between mice when not in proximity
angle_distant = angle.copy()
for i in range(900,len(prox)):
    if prox[i] == 1:
                angle_distant [i] = np.nan
            
# angle between mice when in proximity
angle_in_proximity = angle.copy()
for i in range(900,len(prox)):
    if prox[i] == 0:
                angle_in_proximity [i] = np.nan

# combine all data into one dataframe
dlc_behavior = pd.concat([time, NN_sniff, NN_new, angle, angle_distant, angle_in_proximity], axis=1)
dlc_behavior.columns = ['time','NN_sniff','NN_new','angle','angle_distant','angle_in_proximity']
dlc_behavior.dropna(subset=['time'], inplace=True)

In [ ]:
#add dFF and z-score data from previous analysis to this dataframe
photometry_data = pd.read_csv('Your File Path/Analyzed_Photometry.csv')

complied_data = pd.concat([dlc_behavior, photometry_data[['zscore', 'dFF']]], axis=1)

# save dataframe to csv
complied_data.to_csv('Your File Path/dlc_behavior+photometry.csv', index=False)

In [ ]:
#Chop for +/-10 seconds around each new bout of NN Sniff

ts = complied_data.loc[complied_data['NN_new'] == 1]['time'].tolist() #only rows where prox_new is 1
eventTimes = np.linspace(-10,10,601)

dataOut = {'Time':eventTimes}

for i,time in enumerate(ts):
    individual_events = complied_data.loc[(complied_data['time']> time -10) & (complied_data['time'] < time + 10)]
    
    photometry = individual_events['dFF'].tolist()
    a = np.empty(601)
    a[:] = np.nan
    a[:len(photometry)] = photometry
#     print(len(a))
    dataOut.update({'Bout '+str(i):a})
# dataOut
df_new = pd.DataFrame(dataOut)

#Z-Score data using -10 to -5 seconds before behavior starts as the baseline
baseline = df_new.loc[(df_new['Time'] < -5), :]
pre_mean = baseline.mean()
pre_std = np.std(baseline)
norm = (df_new - pre_mean)/pre_std
norm['Time'] = df_new['Time']
NN_chopped = norm

NN_chopped.to_csv('Your File Path/NN_chopped.csv')

NN_chopped